In [9]:
import opendatasets as od
import pandas as pd
import os
import shutil
import librosa

In [4]:
od.download('https://www.kaggle.com/datasets/makvel/mer500')

Skipping, found downloaded files in "./mer500" (use force=True to force download)


In [5]:
lyrics = pd.read_csv('hindi_songs_lyrics.csv')
lyrics['Song_emotion'] = len(lyrics)*[None]
lyrics['Song_location'] = len(lyrics)*[None]

In [6]:
base_dir = '/content/mer500/'
extracted_dir = '/content/extracted_audio'
os.makedirs(extracted_dir, exist_ok=True)

for folder_name in os.listdir(base_dir):
  folder_path = os.path.join(base_dir, folder_name)
  folder_path = os.path.join(folder_path, folder_name)

  for filename in os.listdir(folder_path):
      if filename.endswith('.wav'):
          song_id = filename
          if str(song_id) in list(lyrics['Song_id']):
            source_path = os.path.join(folder_path, filename)
            destination_path = os.path.join(extracted_dir, filename)
            idx = list(lyrics['Song_id']).index(song_id)
            lyrics.loc[idx, 'Song_emotion'] = folder_name
            lyrics.loc[idx, 'Song_location'] = destination_path
            shutil.copy2(source_path, destination_path)

print("Extraction complete")

Extraction complete


In [7]:
unique_labels = lyrics['Song_emotion'].unique()
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
lyrics['Song_emotion'] = lyrics['Song_emotion'].map(label_to_int)

In [50]:
label_to_int

{'Romantic': 0, 'Devotional': 1, 'Party': 2, 'Happy': 3, 'Sad': 4}

In [8]:
lyrics.head()

,Song_id,Lyrics,Song_name,Song_emotion,Song_location
0,301.wav,रंग दे तू मोहे गेरू। राधे की दिल से है दुआ,Rang De Tu Mohe Gerua,0,/content/extracted_audio/301.wav
1,302.wav,सनम रे सनम रे तू मेरा सनम हुआ रे सनम रे सनम रे,Sanam Re,0,/content/extracted_audio/302.wav
2,327.wav,हम तुम्हारे हैं। तुम्हारे सनम हम तुम्हारे हैं।...,Hum Tumhare Hain Sanam,0,/content/extracted_audio/327.wav
3,304.wav,तेरे होके रहेंगे ओ दिल जिद पे अड़ा है ओ तेरे हो...,Tere Hoke Rahenge,0,/content/extracted_audio/304.wav
4,305.wav,प्यार हुआ पूरी हुई दुआ हमको प्यार हुआ पूरी हुई...,Pyaar Hua,0,/content/extracted_audio/305.wav


In [1]:
!pip install torch --index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install sklearn

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModel.from_pretrained("google/muril-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [18]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

lyrics = lyrics.dropna(subset=['Song_emotion'])
lyrics['embeddings'] = lyrics['Lyrics'].apply(get_embeddings)
lyrics['embeddings'] = lyrics['embeddings'].apply(lambda x: x[0])

# lyrics['embeddings'] = lyrics['embeddings'].apply(lambda x: list(x[0]))

In [20]:
SR = 22050
N_MELS = 128
HOP_LENGTH = 512
DURATION = 5
IMG_HEIGHT = N_MELS
IMG_WIDTH = int((SR * DURATION) / HOP_LENGTH)

def get_mel_spectrogram(file_path, sr=SR, n_mels=N_MELS, hop_length=HOP_LENGTH, duration=DURATION):
    y, _ = librosa.load(file_path, sr=sr, duration=duration)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    if mel_spectrogram_db.shape[1] != IMG_WIDTH:
        mel_spectrogram_db = librosa.util.fix_length(mel_spectrogram_db, size=IMG_WIDTH, axis=1)
    mel_spectrogram_flattened = mel_spectrogram_db.flatten()
    return mel_spectrogram_flattened

lyrics['audio_features'] = lyrics['Song_location'].apply(get_mel_spectrogram)

<ipython-input-20-58d104a901f2>:9: UserWarning: PySoundFile failed. Trying audioread instead.
  y, _ = librosa.load(file_path, sr=sr, duration=duration)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [21]:
lyrics = lyrics.dropna(subset=['audio_features', 'embeddings'])
lyrics['combined_features'] = lyrics.apply(lambda row: np.concatenate((row['embeddings'], row['audio_features'])), axis=1)

In [51]:
X_combined = np.array(lyrics['combined_features'].tolist())
y_combined = np.array(lyrics['Song_emotion'])

X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42, stratify=y_combined)

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [24]:
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear']}
grid_svc = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid_svc.fit(X_train, y_train)
print(grid_svc.best_params_)
svm_pred = grid_svc.predict(X_test)
print(classification_report(y_test, svm_pred))

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.167 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.167 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.167 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.167 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.167 total time=   0.2s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.417 total time=   0.2s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.750 total time=   0.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.583 total time=   0.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.500 total time=   0.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.333 total time=   0.2s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.167 total time=   0.2s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf

In [55]:
param_grid = {'C': [0.1, 1, 10, 100]}
grid_lr = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, refit=True, verbose=3)
grid_lr.fit(X_train, y_train)
print(grid_lr.best_params_)
lr_pred = grid_lr.predict(X_test)
print(classification_report(y_test, lr_pred))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END .............................C=0.1;, score=0.500 total time=   5.1s
[CV 2/5] END .............................C=0.1;, score=0.583 total time=   4.6s
[CV 3/5] END .............................C=0.1;, score=0.750 total time=   4.5s
[CV 4/5] END .............................C=0.1;, score=0.417 total time=   5.3s
[CV 5/5] END .............................C=0.1;, score=0.333 total time=   3.7s
[CV 1/5] END ...............................C=1;, score=0.417 total time=   2.1s
[CV 2/5] END ...............................C=1;, score=0.583 total time=   2.2s
[CV 3/5] END ...............................C=1;, score=0.500 total time=   3.6s
[CV 4/5] END ...............................C=1;, score=0.417 total time=   3.2s
[CV 5/5] END ...............................C=1;, score=0.333 total time=   2.2s
[CV 1/5] END ..............................C=10;, score=0.417 total time=   1.2s
[CV 2/5] END ..............................C=10;,

In [27]:
param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}

grid_rfc = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, refit=True, verbose=3)
grid_rfc.fit(X_train, y_train)

print(grid_rfc.best_params_)
rf_pred = grid_rfc.predict(X_test)
print(classification_report(y_test, rf_pred))

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.417 total time=   0.7s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.250 total time=   0.4s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.417 total time=   0.7s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.167 total time=   0.7s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.167 total time=   0.3s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.417 total time=   0.7s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.500 total time=   0.6s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.250 to

In [49]:
best_model = grid.best_estimator_

int_to_label = {idx: label for label, idx in label_to_int.items()}

for class_label in np.unique(y_test):
    class_indices = np.where(y_test == class_label)[0]
    if len(class_indices) > 0:
        test_index = class_indices[1]

        example_input = X_test[test_index]
        example_prediction = best_model.predict(example_input.reshape(1, -1))[0]
        song_info = lyrics.iloc[lyrics.index[lyrics['combined_features'].apply(lambda x: np.array_equal(x, example_input))].tolist()[0]]

        print("File:", song_info["Song_id"])
        print("Lyrics:", song_info["Lyrics"])
        print(f"Actual Emotion: {int_to_label[class_label]}")
        print(f"Predicted Emotion: {int_to_label[example_prediction]}")
        print("-" * 20)

File: 319.wav
Lyrics: जोर से अलविदा जोर से अलविदा जोर से अलविदा ना कहो अलविदा
Actual Emotion: Romantic
Predicted Emotion: Happy
--------------------
File: 111.wav
Lyrics: गणपति की सेवा मंगल मेवा सेवा से सब विघ्न टरे
Actual Emotion: Devotional
Predicted Emotion: Romantic
--------------------
File: 437.wav
Lyrics: बाबूजी जरा धीरे चलो, बिजली खड़ी यहाँ बिजली खड़ी नैनो में चिंगारिया
Actual Emotion: Party
Predicted Emotion: Party
--------------------
File: 11.wav
Lyrics: तेरा ड्रामा मचाते हंगामा लड़ाए जो नजररिया जुल्म हो गवा तेरे है दीवाने तू मानेगा
Actual Emotion: Happy
Predicted Emotion: Happy
--------------------
File: 217.wav
Lyrics: क्योंकि इतना प्यार तुमको करते हैं हम। क्योंकि इतना प्यार तुमको करते हैं हम। क्या जान लोगे हमारी सनम क्योंकि इतना प्यार तुमको करते हैं हम।
Actual Emotion: Sad
Predicted Emotion: Sad
--------------------


In [53]:
X_audio = np.array(lyrics['audio_features'].tolist())
y_audio = np.array(lyrics['Song_emotion'])
X_audio_train, X_audio_test, y_audio_train, y_audio_test = train_test_split(X_audio, y_audio, test_size=0.2, random_state=42, stratify=y_audio)

param_grid = {'C': [0.1, 1, 10, 100]}
grid_lr_audio = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, refit=True, verbose=3)
grid_lr_audio.fit(X_audio_train, y_audio_train)
print(grid_lr_audio.best_params_)
lr_pred = grid_lr_audio.predict(X_audio_test)
print(classification_report(y_audio_test, lr_pred))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END .............................C=0.1;, score=0.500 total time=   4.9s
[CV 2/5] END .............................C=0.1;, score=0.583 total time=   7.5s
[CV 3/5] END .............................C=0.1;, score=0.750 total time=   4.4s
[CV 4/5] END .............................C=0.1;, score=0.417 total time=   3.9s
[CV 5/5] END .............................C=0.1;, score=0.333 total time=   7.0s
[CV 1/5] END ...............................C=1;, score=0.417 total time=   3.5s
[CV 2/5] END ...............................C=1;, score=0.583 total time=   2.5s
[CV 3/5] END ...............................C=1;, score=0.500 total time=   2.2s
[CV 4/5] END ...............................C=1;, score=0.417 total time=   4.2s
[CV 5/5] END ...............................C=1;, score=0.333 total time=   2.7s
[CV 1/5] END ..............................C=10;, score=0.417 total time=   1.0s
[CV 2/5] END ..............................C=10;,

In [54]:
X_lyrics = np.array(lyrics['embeddings'].tolist())
y_lyrics = np.array(lyrics['Song_emotion'])
X_lyrics_train, X_lyrics_test, y_lyrics_train, y_lyrics_test = train_test_split(X_lyrics, y_lyrics, test_size=0.2, random_state=42, stratify=y_lyrics)

param_grid = {'C': [0.1, 1, 10, 100]}
grid_lr_lyrics = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, refit=True, verbose=3)
grid_lr_lyrics.fit(X_lyrics_train, y_lyrics_train)
print(grid_lr_lyrics.best_params_)
lr_pred = grid_lr_lyrics.predict(X_lyrics_test)
print(classification_report(y_lyrics_test, lr_pred))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END .............................C=0.1;, score=0.333 total time=   0.0s
[CV 2/5] END .............................C=0.1;, score=0.250 total time=   0.0s
[CV 3/5] END .............................C=0.1;, score=0.083 total time=   0.0s
[CV 4/5] END .............................C=0.1;, score=0.333 total time=   0.0s
[CV 5/5] END .............................C=0.1;, score=0.167 total time=   0.0s
[CV 1/5] END ...............................C=1;, score=0.333 total time=   0.0s
[CV 2/5] END ...............................C=1;, score=0.250 total time=   0.0s
[CV 3/5] END ...............................C=1;, score=0.167 total time=   0.0s
[CV 4/5] END ...............................C=1;, score=0.333 total time=   0.0s
[CV 5/5] END ...............................C=1;, score=0.333 total time=   0.0s
[CV 1/5] END ..............................C=10;, score=0.333 total time=   0.0s
[CV 2/5] END ..............................C=10;,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [28]:
def get_mel_spectrogram_unflattened(file_path, sr=SR, n_mels=N_MELS, hop_length=HOP_LENGTH, duration=DURATION):
    y, _ = librosa.load(file_path, sr=sr, duration=duration)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    if mel_spectrogram_db.shape[1] != IMG_WIDTH:
        mel_spectrogram_db = librosa.util.fix_length(mel_spectrogram_db, size=IMG_WIDTH, axis=1)
    return mel_spectrogram_db

lyrics['audio_features_raw'] = lyrics['Song_location'].apply(get_mel_spectrogram_unflattened)

lyrics = lyrics.dropna(subset=['audio_features_raw', 'embeddings'])

<ipython-input-28-59ce59425c9f>:14: UserWarning: PySoundFile failed. Trying audioread instead.
  y, _ = librosa.load(file_path, sr=sr, duration=duration)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [37]:
from tensorflow.keras import layers, models

def build_combined_model(input_audio_shape, input_text_shape, num_classes):
    audio_input = layers.Input(shape=input_audio_shape, name="audio_input")
    x_audio = layers.Reshape((input_audio_shape[0], input_audio_shape[1], 1))(audio_input)
    x_audio = layers.Conv2D(64, (3, 3), activation='relu')(x_audio)
    x_audio = layers.MaxPooling2D((2, 2))(x_audio)
    x_audio = layers.Conv2D(32, (3, 3), activation='relu')(x_audio)
    x_audio = layers.MaxPooling2D((2, 2))(x_audio)
    x_audio = layers.Flatten()(x_audio)

    text_input = layers.Input(shape=input_text_shape, name="text_input")
    x_text = layers.Dense(256, activation='relu')(text_input)
    x_text = layers.Dense(128, activation='relu')(x_text)

    combined = layers.concatenate([x_audio, x_text])

    x = layers.Dense(64, activation='relu')(combined)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)

    output = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=[audio_input, text_input], outputs=output)

    return model

In [38]:
X = lyrics[['audio_features_raw', 'embeddings']]
y = lyrics['Song_emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_audio = np.stack(X_train['audio_features_raw'].values)
X_train_text = np.stack(X_train['embeddings'].values)

X_test_audio = np.stack(X_test['audio_features_raw'].values)
X_test_text = np.stack(X_test['embeddings'].values)

In [39]:
input_audio_shape = (N_MELS, IMG_WIDTH)
input_text_shape = (lyrics['embeddings'].iloc[0].shape[0],)

model = build_combined_model(input_audio_shape, input_text_shape, num_classes=len(unique_labels))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit([X_train_audio, X_train_text], y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.2812 - loss: 40.3666 - val_accuracy: 0.1667 - val_loss: 11.3130
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.1944 - loss: 32.7724 - val_accuracy: 0.1667 - val_loss: 7.8012
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.2361 - loss: 8.6881 - val_accuracy: 0.1667 - val_loss: 1.9489
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.1493 - loss: 3.0454 - val_accuracy: 0.2500 - val_loss: 1.6505
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.2743 - loss: 2.2447 - val_accuracy: 0.1667 - val_loss: 2.1568
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.1944 - loss: 2.4153 - val_accuracy: 0.1667 - val_loss: 1.5957
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.3854 - loss: 1.5363 - val_accuracy: 0.2500 - val_loss: 1.5991
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.3958 - loss: 1.5858 - val_accuracy: 0.3333 - val_loss: 1.5983
Epoch 9/20
2

In [36]:
test_loss, test_acc = model.evaluate([X_test_audio, X_test_text], y_test)
print(f"Test accuracy: {test_acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 0.3333 - loss: 1.7555
Test accuracy: 0.3333333432674408
